In [ ]:
!pip install pdfplumber

In [ ]:
!apt-get install -y tesseract-ocr
!pip install pytesseract pdfplumber

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!apt-get install -y tesseract-ocr
!pip install pytesseract pdfplumber pdf2image openpyxl

import pdfplumber
import pytesseract
from PIL import Image
import pandas as pd
import re
import os
from pdf2image import convert_from_path

# Function to check if the file is an image or a PDF
def is_image(file_path):
    image_extensions = ['.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif']
    _, ext = os.path.splitext(file_path)
    return ext.lower() in image_extensions

# Function to extract text and tables from PDF
def extract_text_and_tables_from_pdf(pdf_path):
    text = ""
    tables = {}
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            extracted_text = page.extract_text()
            if extracted_text:
                text += extracted_text
            else:
                # Handle scanned PDF page
                text += extract_text_from_image(convert_pdf_page_to_image(page))

            # Extract tables
            page_tables = page.extract_tables()
            if page_tables:
                tables[f'Page_{page_num + 1}'] = page_tables
    return text, tables

# Function to extract text from an image using Tesseract
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

# Function to convert PDF page to image
def convert_pdf_page_to_image(pdf_page):
    image = pdf_page.to_image(resolution=300)
    return image.original

# Function to clean text from illegal characters
def clean_text(text):
    return ''.join(c for c in text if c.isprintable())

# Function to find the required information using regex and capture surrounding text
def find_information(text, tables):
    info = {}
    surrounding_context = {}

    # Define regex patterns for different terminologies
    patterns = {
        'contract_number': [
            r'Contract Number:\s*(.*)', r'Contract ID:\s*(.*)', r'Agreement Number:\s*(.*)',
            r'Contract No\.\s*(.*)', r'Contract Reference:\s*(.*)', r'Contract Code:\s*(.*)',
            r'Agreement ID:\s*(.*)', r'Agreement No\.\s*(.*)', r'Contract Identification Number:\s*(.*)'
        ],
        'start_date': [
            r'entered into as of\s*(\w+ \d+, \d+)', r'Start Date:\s*(\w+ \d+, \d+)', r'Effective Date:\s*(\w+ \d+, \d+)',
            r'Commencement Date:\s*(\w+ \d+, \d+)', r'Agreement Start Date:\s*(\w+ \d+, \d+)', r'Execution Date:\s*(\w+ \d+, \d+)',
            r'Beginning Date:\s*(\w+ \d+, \d+)', r'Date of Commencement:\s*(\w+ \d+, \d+)', r'Contract Start Date:\s*(\w+ \d+, \d+)'
        ],
        'end_date': [
            r'terminating on\s*(\w+ \d+, \d+)', r'End Date:\s*(\w+ \d+, \d+)', r'Expiration Date:\s*(\w+ \d+, \d+)',
            r'Contract End Date:\s*(\w+ \d+, \d+)', r'Agreement End Date:\s*(\w+ \d+, \d+)', r'Termination Date:\s*(\w+ \d+, \d+)',
            r'Completion Date:\s*(\w+ \d+, \d+)', r'Finish Date:\s*(\w+ \d+, \d+)', r'Closure Date:\s*(\w+ \d+, \d+)'
        ],
        'supplier_name': [
            r'Party B:[\s\S]+Name:\s*(.*)', r'Supplier:\s*(.*)', r'Vendor Name:\s*(.*)',
            r'Contractor:\s*(.*)', r'Service Provider:\s*(.*)', r'Provider Name:\s*(.*)',
            r'Supplier Name:\s*(.*)', r'Vendor:\s*(.*)', r'Contractor Name:\s*(.*)'
        ],
        'invoice_id': [
            r'unique invoice number, starting with\s*(\w+-\d+)', r'Invoice ID:\s*(\w+-\d+)', r'Billing ID:\s*(\w+-\d+)',
            r'Invoice Number:\s*(\w+-\d+)', r'Bill Number:\s*(\w+-\d+)', r'Invoice Reference:\s*(\w+-\d+)',
            r'Billing Number:\s*(\w+-\d+)', r'Invoice No\.\s*(\w+-\d+)', r'Bill ID:\s*(\w+-\d+)'
        ],
        'amount_of_contract': [
            r'Total Contract Amount:\s*\$(.*)', r'Contract Value:\s*\$(.*)', r'Amount:\s*\$(.*)',
            r'Total Amount:\s*\$(.*)', r'Contract Sum:\s*\$(.*)', r'Total Value:\s*\$(.*)',
            r'Contract Price:\s*\$(.*)', r'Value of Contract:\s*\$(.*)', r'Total Contract Value:\s*\$(.*)'
        ]
    }

    # Function to apply regex patterns
    def apply_patterns(pattern_list, text):
        for pattern in pattern_list:
            match = re.search(pattern, text)
            if match:
                return match.group(1).strip()
        return "Not found"

    # Extract information using the patterns
    info['contract_number'] = apply_patterns(patterns['contract_number'], text)
    info['start_date'] = apply_patterns(patterns['start_date'], text)
    info['end_date'] = apply_patterns(patterns['end_date'], text)
    info['supplier_name'] = apply_patterns(patterns['supplier_name'], text)
    info['invoice_id'] = apply_patterns(patterns['invoice_id'], text)
    info['amount_of_contract'] = apply_patterns(patterns['amount_of_contract'], text)

    # Extracting KPIs
    kpi_pattern = re.compile(r'KPIs:[\s\S]+?(?=Reporting Requirements|Miscellaneous)')
    kpis = kpi_pattern.search(text)
    if kpis:
        kpi_text = kpis.group(0).strip()
        info['KPIs'] = kpi_text

        # Extract tables in KPIs section
        for page_num, page_tables in tables.items():
            for table in page_tables:
                table_text = ' '.join([' '.join(row) for row in table])
                if 'KPIs' in table_text:
                    info[f'KPIs_table_{page_num}'] = table

    else:
        info['KPIs'] = "Not found"

    # Extracting Reporting Requirements
    reporting_pattern = re.compile(r'Reporting Requirements:[\s\S]+?(?=Miscellaneous)')
    reporting_requirements = reporting_pattern.search(text)
    if reporting_requirements:
        reporting_text = reporting_requirements.group(0).strip()
        info['reporting_requirements'] = reporting_text

        # Extract tables in Reporting Requirements section
        for page_num, page_tables in tables.items():
            for table in page_tables:
                table_text = ' '.join([' '.join(row) for row in table])
                if 'Reporting Requirements' in table_text:
                    info[f'Reporting_requirements_table_{page_num}'] = table
    else:
        info['reporting_requirements'] = "Not found"

    # Extracting Scope of Services
    scope_pattern = re.compile(r'Scope\s*of\s*Services\s*:[\s\S]+?(?=Term\s*and\s*Termination|$)')
    scope_of_services_match = scope_pattern.search(text)
    if scope_of_services_match:
        scope_of_services_text = scope_of_services_match.group(0).strip()

        # Extract subheadings and bullet points
        subheadings_pattern = re.compile(r'([A-Z][\w\s]*):\s*([\s\S]+?)(?=[A-Z][\w\s]*:|$)')
        subheadings = subheadings_pattern.findall(scope_of_services_text)

        subheadings_dict = {}
        for subheading in subheadings:
            subheadings_dict[subheading[0]] = subheading[1].strip().split('\n')

        info['scope_of_services'] = subheadings_dict
    else:
        scope_start = text.find("Scope of Services")
        surrounding_context['scope_of_services_context'] = text[scope_start:scope_start + 1000]
        info['scope_of_services'] = "Not found"

    return info, surrounding_context

# Function to save the extracted information to an Excel file
def save_to_excel(all_info, all_surrounding_contexts, all_tables, excel_path):
    data = []
    for info, surrounding_context in zip(all_info, all_surrounding_contexts):
        data.append({**info, **surrounding_context})

    df = pd.DataFrame(data)

    # Clean illegal characters in all dataframe cells
    for column in df.columns:
        df[column] = df[column].apply(lambda x: clean_text(str(x)))

    df.to_excel(excel_path, index=False)

    # Save tables separately
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
        for pdf_idx, tables in enumerate(all_tables):
            for key, value in tables.items():
                for table_idx, table in enumerate(value):
                    table_df = pd.DataFrame(table)
                    sheet_name = f'PDF{pdf_idx + 1}_{key}_Table{table_idx + 1}'
                    table_df.to_excel(writer, sheet_name=sheet_name, index=False)

# Main execution
pdf_files = ['/content/Sample Contract.pdf', '/content/Sample Contract.pdf']  # Add your file paths
excel_path = '/content/Save_info.xlsx'

all_info = []
all_surrounding_contexts = []
all_tables = []

for file_path in pdf_files:
    if is_image(file_path):
        text = extract_text_from_image(file_path)
        tables = {}  # No tables in images for now
    else:
        text, tables = extract_text_and_tables_from_pdf(file_path)

    info, surrounding_context = find_information(text, tables)
    all_info.append(info)
    all_surrounding_contexts.append(surrounding_context)
    all_tables.append(tables)

save_to_excel(all_info, all_surrounding_contexts, all_tables, excel_path)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!apt-get install -y tesseract-ocr
!pip install pytesseract pdfplumber pdf2image openpyxl

import pdfplumber
import pytesseract
from PIL import Image
import pandas as pd
import re
import os
from pdf2image import convert_from_path

# Function to check if the file is an image or a PDF
def is_image(file_path):
    image_extensions = ['.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif']
    _, ext = os.path.splitext(file_path)
    return ext.lower() in image_extensions

# Function to extract text and tables from PDF
def extract_text_and_tables_from_pdf(pdf_path):
    text = ""
    tables = {}
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            extracted_text = page.extract_text()
            if extracted_text:
                text += extracted_text
            else:
                # Handle scanned PDF page
                text += extract_text_from_image(convert_pdf_page_to_image(page))

            # Extract tables
            page_tables = page.extract_tables()
            if page_tables:
                tables[f'Page_{page_num + 1}'] = page_tables
    return text, tables

# Function to extract text from an image using Tesseract
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

# Function to convert PDF page to image
def convert_pdf_page_to_image(pdf_page):
    image = pdf_page.to_image(resolution=300)
    return image.original

# Function to clean text from illegal characters
def clean_text(text):
    return ''.join(c for c in text if c.isprintable())

# Function to find the required information using regex and capture surrounding text
def find_information(text, tables):
    info = {}
    surrounding_context = {}

    # Define regex patterns for different terminologies
    patterns = {
        'contract_number': [
            r'Contract Number:\s*(.*)', r'Contract ID:\s*(.*)', r'Agreement Number:\s*(.*)',
            r'Contract No\.\s*(.*)', r'Contract Reference:\s*(.*)', r'Contract Code:\s*(.*)',
            r'Agreement ID:\s*(.*)', r'Agreement No\.\s*(.*)', r'Contract Identification Number:\s*(.*)'
        ],
        'start_date': [
            r'entered into as of\s*(\w+ \d+, \d+)', r'Start Date:\s*(\w+ \d+, \d+)', r'Effective Date:\s*(\w+ \d+, \d+)',
            r'Commencement Date:\s*(\w+ \d+, \d+)', r'Agreement Start Date:\s*(\w+ \d+, \d+)', r'Execution Date:\s*(\w+ \d+, \d+)',
            r'Beginning Date:\s*(\w+ \d+, \d+)', r'Date of Commencement:\s*(\w+ \d+, \d+)', r'Contract Start Date:\s*(\w+ \d+, \d+)'
        ],
        'end_date': [
            r'terminating on\s*(\w+ \d+, \d+)', r'End Date:\s*(\w+ \d+, \d+)', r'Expiration Date:\s*(\w+ \d+, \d+)',
            r'Contract End Date:\s*(\w+ \d+, \d+)', r'Agreement End Date:\s*(\w+ \d+, \d+)', r'Termination Date:\s*(\w+ \d+, \d+)',
            r'Completion Date:\s*(\w+ \d+, \d+)', r'Finish Date:\s*(\w+ \d+, \d+)', r'Closure Date:\s*(\w+ \d+, \d+)'
        ],
        'supplier_name': [
            r'Party B:[\s\S]+Name:\s*(.*)', r'Supplier:\s*(.*)', r'Vendor Name:\s*(.*)',
            r'Contractor:\s*(.*)', r'Service Provider:\s*(.*)', r'Provider Name:\s*(.*)',
            r'Supplier Name:\s*(.*)', r'Vendor:\s*(.*)', r'Contractor Name:\s*(.*)'
        ],
        'invoice_id': [
            r'unique invoice number, starting with\s*(\w+-\d+)', r'Invoice ID:\s*(\w+-\d+)', r'Billing ID:\s*(\w+-\d+)',
            r'Invoice Number:\s*(\w+-\d+)', r'Bill Number:\s*(\w+-\d+)', r'Invoice Reference:\s*(\w+-\d+)',
            r'Billing Number:\s*(\w+-\d+)', r'Invoice No\.\s*(\w+-\d+)', r'Bill ID:\s*(\w+-\d+)'
        ],
        'amount_of_contract': [
            r'Total Contract Amount:\s*\$(.*)', r'Contract Value:\s*\$(.*)', r'Amount:\s*\$(.*)',
            r'Total Amount:\s*\$(.*)', r'Contract Sum:\s*\$(.*)', r'Total Value:\s*\$(.*)',
            r'Contract Price:\s*\$(.*)', r'Value of Contract:\s*\$(.*)', r'Total Contract Value:\s*\$(.*)'
        ]
    }

    # Function to apply regex patterns
    def apply_patterns(pattern_list, text):
        for pattern in pattern_list:
            match = re.search(pattern, text)
            if match:
                return match.group(1).strip()
        return "Not found"

    # Extract information using the patterns
    info['contract_number'] = apply_patterns(patterns['contract_number'], text)
    info['start_date'] = apply_patterns(patterns['start_date'], text)
    info['end_date'] = apply_patterns(patterns['end_date'], text)
    info['supplier_name'] = apply_patterns(patterns['supplier_name'], text)
    info['invoice_id'] = apply_patterns(patterns['invoice_id'], text)
    info['amount_of_contract'] = apply_patterns(patterns['amount_of_contract'], text)

    # Extracting KPIs
    kpi_pattern = re.compile(r'KPIs:[\s\S]+?(?=Reporting Requirements|Miscellaneous)')
    kpis = kpi_pattern.search(text)
    if (kpis):
        kpi_text = kpis.group(0).strip()
        info['KPIs'] = kpi_text

        # Extract tables in KPIs section
        for page_num, page_tables in tables.items():
            for table in page_tables:
                table_text = ' '.join([' '.join(row) for row in table])
                if 'KPIs' in table_text:
                    info[f'KPIs_table_{page_num}'] = table
    else:
        info['KPIs'] = "Not found"

    # Extracting Reporting Requirements
    reporting_pattern = re.compile(r'Reporting Requirements:[\s\S]+?(?=Miscellaneous)')
    reporting_requirements = reporting_pattern.search(text)
    if reporting_requirements:
        reporting_text = reporting_requirements.group(0).strip()
        info['reporting_requirements'] = reporting_text

        # Extract tables in Reporting Requirements section
        for page_num, page_tables in tables.items():
            for table in page_tables:
                table_text = ' '.join([' '.join(row) for row in table])
                if 'Reporting Requirements' in table_text:
                    info[f'Reporting_requirements_table_{page_num}'] = table
    else:
        info['reporting_requirements'] = "Not found"

    # Extracting Scope of Services
    scope_pattern = re.compile(r'Scope\s*of\s*Services\s*:[\s\S]+?(?=Term\s*and\s*Termination|$)')
    scope_of_services_match = scope_pattern.search(text)
    if scope_of_services_match:
        scope_of_services_text = scope_of_services_match.group(0).strip()

        # Extract subheadings and bullet points
        subheadings_pattern = re.compile(r'([A-Z][\w\s]*):\s*([\s\S]+?)(?=[A-Z][\w\s]*:|$)')
        subheadings = subheadings_pattern.findall(scope_of_services_text)

        subheadings_dict = {}
        for subheading in subheadings:
            subheadings_dict[subheading[0]] = subheading[1].strip().split('\n')

        info['scope_of_services'] = subheadings_dict
    else:
        scope_start = text.find("Scope of Services")
        surrounding_context['scope_of_services_context'] = text[scope_start:scope_start + 1000]
        info['scope_of_services'] = "Not found"

    return info, surrounding_context

# Function to save the extracted information to an Excel file
def save_to_excel(all_info, all_surrounding_contexts, all_tables, excel_path):
    data = []
    for info, surrounding_context in zip(all_info, all_surrounding_contexts):
        data.append({**info, **surrounding_context})

    df = pd.DataFrame(data)

    # Clean illegal characters in all dataframe cells
    for column in df.columns:
        df[column] = df[column].apply(lambda x: clean_text(str(x)))

    df.to_excel(excel_path, index=False)

    # Save tables separately
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a') as writer:
        for pdf_idx, tables in enumerate(all_tables):
            for key, value in tables.items():
                for table_idx, table in enumerate(value):
                    table_df = pd.DataFrame(table)
                    sheet_name = f'PDF{pdf_idx + 1}_{key}_Table{table_idx + 1}'
                    table_df.to_excel(writer, sheet_name=sheet_name, index=False)

# Main execution
folder_path = '/content/your_folder_name'  # Specify your folder path
excel_path = '/content/Save_info.xlsx'

# List all files in the folder
pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.pdf', '.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]

all_info = []
all_surrounding_contexts = []
all_tables = []

for file_path in pdf_files:
    if is_image(file_path):
        image = Image.open(file_path)
        text = extract_text_from_image(image)
        tables = {}  # No tables in images for now
    else:
        text, tables = extract_text_and_tables_from_pdf(file_path)

    info, surrounding_context = find_information(text, tables)
    all_info.append(info)
    all_surrounding_contexts.append(surrounding_context)
    all_tables.append(tables)

save_to_excel(all_info, all_surrounding_contexts, all_tables, excel_path)